In [1]:
# from metrics.utils import hidden_states_collapse
# from metrics.query import DataFrameQuery
# from common.tensor_storage import TensorStorage
# from common.metadata_db import MetadataDB
# from metrics.utils import  exact_match, angular_distance
# from common.utils import *

#from sklearn.feature_selection import mutual_info_regression MISSIN?
from dadapy.data import Data

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
# from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer

from pathlib import Path
import pickle


# Cosine Similarity Debug

### Setup

In [16]:
def angular_distance(mat):
    """
    Computes distance based on angles between vectors, over the rows of the matrix
    """
    dot_product = mat @ mat.T
    norm_vector = np.linalg.norm(mat, axis=1)
    stacked_vector = np.tile(norm_vector, (mat.shape[0], 1))
    norm_product = stacked_vector.T*stacked_vector

    cosine_similarity = dot_product / norm_product
    cosine_similarity = np.clip(cosine_similarity, -1, 1)
    distances = np.arccos(cosine_similarity) / np.pi
    return distances

In [3]:
def angular_distance_v(v1, v2):
    """Compute the angular distance between two vectors."""
    dot_product = np.dot(v1, v2)
    norm_product = np.linalg.norm(v1) * np.linalg.norm(v2)
    cosine_similarity = dot_product / norm_product
    # Ensure the value falls within the valid domain for arccos, which is [-1, 1]
    cosine_similarity = np.clip(cosine_similarity, -1, 1)
    #assert cosine_similarity == cosine_similarity1 # Ensure the value is not out of bounds
    angle = np.arccos(cosine_similarity) #could they be negative?
    return angle / np.pi

In [4]:
def softmax(x):
    exp_values = np.exp(x)# - np.max(x, axis=-1, keepdims=True))
    probabilities = exp_values / np.sum(exp_values, axis=-1, keepdims=True)
    return probabilities

In [5]:
def rotate_vector(vector, angle_degrees):
    # Convert angle from degrees to radians
    angle_radians = np.radians(angle_degrees)
    
    # Rotation matrix
    rotation_matrix = np.array([
        [np.cos(angle_radians), -np.sin(angle_radians)],
        [np.sin(angle_radians), np.cos(angle_radians)]
    ])
    
    rotated_vector = np.dot(rotation_matrix, vector)
    
    return rotated_vector

In [6]:
def cosine_distance(v1, v2):
    """Compute the cosine distance between two vectors."""
    similarity = spatial.distance.cosine(v1, v2)
    return similarity

## Toy Test

In [7]:
vi = np.random.rand(10,100)*100
vj = np.random.rand(10,100)*100

In [8]:
vi_norm = vi/np.linalg.norm(vi)
vj_norm = vj/np.linalg.norm(vj)

In [22]:
dist_i = angular_distance(vi)
dist_j = angular_distance(vj)
data_ad = Data(distances=dist_i)
data_ad.compute_neigh_indices()
overlap_ad = data_ad.return_data_overlap(distances=dist_j, k=5)

/orfeo/cephfs/home/dssc/zenocosini/helm_suite/DADApy/dadapy/kstar.py:101: UserWarning: Careful! The intrinsic dimension is not defined. Computing it unsupervisedly with 'compute_id_2NN()' method
  warnings.warn(


In [23]:
print(f"{data_ad.dist_indices=}")

data_ad.dist_indices=array([[0, 2, 9, 3, 8, 6, 7, 1, 5, 4],
       [1, 9, 5, 4, 6, 3, 2, 7, 0, 8],
       [2, 6, 8, 0, 1, 5, 4, 9, 7, 3],
       [3, 1, 6, 9, 5, 0, 7, 2, 4, 8],
       [4, 1, 6, 7, 5, 2, 8, 3, 0, 9],
       [5, 6, 8, 1, 7, 9, 4, 2, 3, 0],
       [6, 8, 5, 7, 2, 4, 1, 3, 9, 0],
       [7, 6, 4, 5, 1, 8, 2, 3, 0, 9],
       [8, 6, 5, 2, 9, 7, 4, 0, 1, 3],
       [9, 1, 0, 5, 8, 3, 6, 2, 7, 4]])


In [20]:
data_n = Data(vi_norm)
data_n.compute_distances()
overlap_n = data_n.return_data_overlap(vj_norm, k=5)

In [21]:
print(f"{data_n.dist_indices=}")

data_n.dist_indices=array([[0, 9, 2, 3, 8, 7, 6, 5, 1, 4],
       [1, 5, 9, 4, 3, 7, 6, 2, 0, 8],
       [2, 8, 0, 6, 5, 4, 1, 9, 7, 3],
       [3, 9, 1, 6, 5, 0, 7, 4, 8, 2],
       [4, 7, 5, 1, 6, 8, 2, 3, 0, 9],
       [5, 8, 6, 7, 1, 9, 4, 3, 2, 0],
       [6, 8, 5, 7, 4, 2, 3, 1, 9, 0],
       [7, 6, 5, 4, 1, 8, 3, 0, 2, 9],
       [8, 5, 6, 2, 9, 7, 4, 0, 3, 1],
       [9, 0, 1, 5, 8, 3, 6, 2, 7, 4]])


In [169]:
nbrs = NearestNeighbors(n_neighbors=9, metric=cosine_distance, n_jobs=8).fit(vi)
distances, dist_indices = nbrs.kneighbors(vi)

In [170]:
dist_indices

array([[0, 9, 2, 5, 7, 4, 3, 8, 1],
       [1, 3, 9, 2, 5, 6, 8, 4, 0],
       [2, 9, 4, 3, 0, 8, 1, 5, 6],
       [3, 6, 9, 4, 1, 2, 5, 7, 0],
       [4, 6, 5, 3, 2, 0, 9, 7, 8],
       [5, 4, 9, 3, 0, 6, 8, 1, 2],
       [6, 8, 4, 9, 3, 7, 5, 1, 2],
       [7, 9, 6, 8, 0, 3, 4, 1, 2],
       [8, 6, 9, 7, 4, 3, 2, 0, 5],
       [9, 2, 6, 7, 8, 5, 3, 0, 1]])

In [126]:
mat = vi
dot_product = np.matmul(mat,mat.T)
norm_vector = np.linalg.norm(mat, axis=1)
stacked_vector = np.tile(norm_vector, (mat.shape[0], 1))
norm_product = stacked_vector.T*stacked_vector

cosine_similarity = np.divide(dot_product,norm_product)
cosine_similarity = np.clip(cosine_similarity, -1, 1)
distances = np.arccos(cosine_similarity) / np.pi

In [127]:
np.where(cosine_similarity > 1)

(array([], dtype=int64), array([], dtype=int64))

In [129]:
distances

array([[6.71e-09, 2.47e-01, 2.59e-01, 2.46e-01, 2.23e-01, 2.48e-01,
        2.63e-01, 2.66e-01, 2.54e-01, 2.33e-01],
       [2.47e-01, 4.74e-09, 2.20e-01, 2.31e-01, 2.03e-01, 2.26e-01,
        2.23e-01, 2.75e-01, 2.40e-01, 1.91e-01],
       [2.59e-01, 2.20e-01, 6.71e-09, 2.57e-01, 2.50e-01, 2.43e-01,
        2.22e-01, 2.53e-01, 2.31e-01, 2.21e-01],
       [2.46e-01, 2.31e-01, 2.57e-01, 0.00e+00, 2.36e-01, 2.48e-01,
        2.40e-01, 2.32e-01, 2.14e-01, 2.24e-01],
       [2.23e-01, 2.03e-01, 2.50e-01, 2.36e-01, 0.00e+00, 2.42e-01,
        2.30e-01, 2.37e-01, 2.31e-01, 2.34e-01],
       [2.48e-01, 2.26e-01, 2.43e-01, 2.48e-01, 2.42e-01, 0.00e+00,
        2.49e-01, 2.92e-01, 2.62e-01, 2.24e-01],
       [2.63e-01, 2.23e-01, 2.22e-01, 2.40e-01, 2.30e-01, 2.49e-01,
        0.00e+00, 2.74e-01, 2.49e-01, 2.31e-01],
       [2.66e-01, 2.75e-01, 2.53e-01, 2.32e-01, 2.37e-01, 2.92e-01,
        2.74e-01, 0.00e+00, 2.49e-01, 2.61e-01],
       [2.54e-01, 2.40e-01, 2.31e-01, 2.14e-01, 2.31e-01, 2.62e-

## Test periodicity

In [73]:
v = np.random.rand(2,2)

In [78]:
vo = rotate_vector(v[0],180)

In [83]:
v_a = np.stack([v[0],vo])

In [88]:
mat = v_a
dot_product = mat @ mat.T
norm_vector = np.linalg.norm(mat, axis=1)
stacked_vector = np.tile(norm_vector, (mat.shape[0], 1))
norm_product = stacked_vector.T*stacked_vector

cosine_similarity = dot_product / norm_product
cosine_similarity = np.clip(cosine_similarity, -1, 1)
distances = np.arccos(cosine_similarity) / np.pi

In [90]:
distances

array([[0., 1.],
       [1., 0.]])

In [143]:
np.cos(v[0],vo)

array([0.76, 0.98])

In [160]:
from scipy import spatial

spatial.distance.cosine(np.zeros(3), np.zeros(3))

/u/dssc/zenocosini/miniconda3/envs/mcqa/lib/python3.11/site-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


0